In [14]:
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, label_binarize, MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

# Cargar datos y realizar preprocesamiento
data = pd.read_excel('ramen-ratings.xlsx')
data = data.drop(columns='Top Ten')
data = pd.get_dummies(data, columns=['Brand', 'Variety', 'Style', 'Country'])
data.replace('Unrated', pd.NA, inplace=True)

# Convertir el DataFrame a tipo numérico
data_numerico = data.apply(pd.to_numeric, errors='coerce')

# Eliminar las filas que contienen valores no finitos (NaN o inf)
data_numerico = data_numerico.dropna()

# Convertir a tipo entero
data_numerico = data_numerico.astype(int)

# Asignar el DataFrame convertido de nuevo a 'data'
data = data_numerico

# Dividir los datos en conjuntos de entrenamiento y prueba
X = data.drop('Stars', axis=1)
y = data['Stars']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar modelos de clasificación: Regresión Logística y Support Vector Machine (SVM)
# Regresión Logística
logistic_regression = LogisticRegression(max_iter=1000)
logistic_regression.fit(X_train_scaled, y_train)

# Support Vector Machine (SVM)
svm = SVC(probability=True)
svm.fit(X_train_scaled, y_train)

# Realizar una búsqueda de hiperparámetros para optimizar el rendimiento de los modelos
# Hiperparámetros para Regresión Logística
logistic_params = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Hiperparámetros para SVM
svm_params = {'C': [0.1, 1, 10], 'gamma': [0.1, 0.01, 0.001], 'kernel': ['rbf', 'linear']}

# Realizar la búsqueda de hiperparámetros
logistic_grid_search = GridSearchCV(LogisticRegression(max_iter=2000), param_grid=logistic_params, cv=5)
logistic_grid_search.fit(X_train_scaled, y_train)

svm_grid_search = GridSearchCV(SVC(), param_grid=svm_params, cv=5)
svm_grid_search.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros y rendimiento de los modelos
print("Mejores hiperparámetros para Regresión Logística:", logistic_grid_search.best_params_)
print("Mejores hiperparámetros para SVM:", svm_grid_search.best_params_)

# Evaluar el rendimiento de los modelos en el conjunto de prueba
logistic_predictions = logistic_regression.predict(X_test_scaled)
svm_predictions = svm.predict(X_test_scaled)

print("Informe de clasificación para Regresión Logística:")
print(classification_report(y_test, logistic_predictions))

print("Informe de clasificación para SVM:")
print(classification_report(y_test, svm_predictions))

# Calcular AUC-ROC para Regresión Logística
logistic_probabilities = logistic_regression.predict_proba(X_test_scaled)
mlb = MultiLabelBinarizer().fit(y_test.values.reshape(-1,1))
y_test_binarized = mlb.transform(y_test.values.reshape(-1,1)).ravel()
logistic_positive_probabilities = logistic_probabilities[:, 1]
logistic_auc_roc = roc_auc_score(y_test_binarized, logistic_positive_probabilities, average='weighted', multi_class='ovr')

# Calcular AUC-ROC para SVM
svm_probabilities = svm.predict_proba(X_test_scaled)
svm_auc_roc = roc_auc_score(y_test, svm_probabilities, average='weighted', multi_class='ovr')

# Mostrar resultados
print("\nResultados para Regresión Logística:")
print("AUC-ROC promedio ponderada:", logistic_auc_roc)

print("\nResultados para SVM:")
print("AUC-ROC promedio ponderada:", svm_auc_roc)
